In [1]:
import numpy as np
import pandas as pd
#col_array = ['label', 'label2' 'label_id', 'speed', 'heading_x', 'heading_y', 'heading_z', 'acceleration_x', 'acceleration_y', 'acceleration_z', 'gyroscope_x', 'gyroscope_y', 'gyroscope_z', 'magnetic_field_x', 'magnetic_field_y', 'magnetic_field_z', 'pressure_hPa', 'log_timestamp']
col_array = ['label', 'acceleration_x', 'acceleration_y', 'acceleration_z', 'gyroscope_x', 'gyroscope_y', 'gyroscope_z', 'magnetic_field_x', 'magnetic_field_y', 'magnetic_field_z']
path = "posmo/experiments/data/posmo_segments/v8/" 
def read_values(csv, col_array, path="posmo/experiments/data/posmo_segments/v8/"):
    return pd.read_csv(path + csv).loc[:, col_array] 

In [2]:
import tensorflow as tf

In [3]:
bike1 = read_values("Bike-20180425T600 PM-aelvyr@gmail.com-8DD36F40-3778-454F-A8D8-E330C6F39830.csv", col_array)
bike2 = read_values("Bike-20180426T518 PM-aelvyr@gmail.com-33A201D1-213B-4B6B-A555-84A9CF015E87.csv", col_array)
bike3 = read_values("Bike-20180426T527 PM-aelvyr@gmail.com-8623C55E-8B27-4D36-93C5-7F91F53CFF6C.csv", col_array)
bike4 = read_values("Bike-20180427T149 PM-aelvyr@gmail.com-1222C4CE-8523-45AD-A22B-030C80EEEDDC.csv", col_array)
bike5 = read_values("Bike-20180427T1218 AM-aelvyr@gmail.com-09521095-A9D3-4662-A914-7DCBC1422947.csv", col_array)
bus1 = read_values("Bus-20180423T0929-djordje@datamap.io-F068A314-DC4D-4FB0-9D97-E79929993E41.csv", col_array)
bus2 = read_values("Bus-20180423T2150-djordje@datamap.io-DEB56810-DB1E-4E14-A144-130B52E0F405.csv", col_array)
car1 = read_values("Car-20180420T0000-djordje@datamap.io-B2876225-A37C-4A72-B045-790E5437B114.csv", col_array)
car2 = read_values("Car-20180420T0912-djordje@datamap.io-37B35625-AAF7-4217-9535-38D6BCF3FF93.csv", col_array)
car3 = read_values("Car-20180420T1328-roger@datamap.io-A3DB32B6-A6AC-4BD9-88EF-8AFFCFCCEDC1.csv", col_array)
flat1 = read_values("Other-Flat-20180424T1445-roger@datamap.io-1A6D2916-7C62-48D4-8CB9-9CBBA9FD7E20.csv", col_array)
flat2 = read_values("Other-flat-reversed-20180424T1538-roger@datamap.io-7419A222-F45B-415A-A3DF-08691E85B99A.csv", col_array)
standing = read_values("Other-InPocket-20180424T1446-roger@datamap.io-6DA03A16-DE1F-4508-BFC8-E8403F621954.csv", col_array)
scooter = read_values("Other-Scooter-20180422T2049-roger@datamap.io-0C33D54C-1B7B-4C45-92D3-2F89A7A82C16.csv", col_array)
tram1 = read_values("Tram-20180426T1825-roger@datamap.io-7C72A146-5733-446A-8418-5126D1BB74DE.csv", col_array)
tram2 = read_values("Tram-20180427T1806-roger@datamap.io-9D3D94ED-4E6B-4382-87A0-0F9A3BA4AA82.csv", col_array)
train = read_values("Train-20180420T1731-roger@datamap.io-E15B0F2F-910B-49CC-A96D-D54276492C97.csv", col_array)
walk1 = read_values("Walk-20180418T1004-roger@datamap.io-D7CA4A5B-0BDC-4E28-9BF4-3DD46B6F91AC.csv", col_array)
walk2 = read_values("Walk-20180418T1507-roger@datamap.io-F3AD9D9D-6B43-4AD1-885B-EAFF3E531A25.csv", col_array)
walk3 = read_values("Walk-20180418T1753-roger@datamap.io-AE01B243-595E-4F19-BAE3-8068038A1AE0.csv", col_array)
walk4 = read_values("Walk-20180419T0011-roger@datamap.io-50A0AB4B-4DB1-4231-8BA6-179DF3EFA381.csv", col_array)
walk5 = read_values("Walk-20180419T1025-roger@datamap.io-9F5D6118-143B-4625-8CD2-A94C5E7E62A0.csv", col_array)


In [4]:
label_dict = {'Bike': 0, 'Bus': 1, 'Car': 2, 'Tram': 3, 'Train': 4, 'Walk': 5}

In [5]:
def feature_extraction(df, f_train, l_train, f_test, l_test, seq_len=250):
    df_np = df.as_matrix()
    df_np = df_np[:(df_np.shape[0] - df_np.shape[0] % seq_len), :]
    df_np = df_np.reshape(int(df_np.shape[0]/seq_len), seq_len, df_np.shape[1])
    for i in range(df_np.shape[0]):
        label = label_dict[df_np[i, 0, 0]]
        feature = np.expand_dims(df_np[i, :, 1:10], axis=0)
        if i % 10 == 0:
            l_test = np.append(l_test, label)
            f_test = np.concatenate([f_test, feature], axis=0)
        else:
            l_train = np.append(l_train, label)
            f_train = np.concatenate([f_train, feature], axis=0)
    #feature = df_np[:, :, 1:18]
    #len = feature.shape[0]
    #n = int((len - len % 10) / 10)
    #feature_test = df_np[:n, :, 1:18]
    #feature_train = df_np[n:, :, 1:18]
    #f_test = np.concatenate([f_test, feature_test], axis=0)
    #f_train = np.concatenate([f_train, feature_train], axis=0)
    return f_train, l_train, f_test, l_test

In [6]:
def return_shapes():
    return f_train.shape, l_train.shape, f_test.shape, l_test.shape

In [7]:
f_train = np.empty([1, 250, 9])
l_train = np.array([])
f_test = np.empty([1, 250, 9])
l_test = np.array([])
f_train, l_train, f_test, l_test = feature_extraction(bike1, f_train, l_train, f_test, l_test)
f_train, l_train, f_test, l_test = feature_extraction(bike2, f_train, l_train, f_test, l_test)
f_train, l_train, f_test, l_test = feature_extraction(bike3, f_train, l_train, f_test, l_test)
f_train, l_train, f_test, l_test = feature_extraction(bike4, f_train, l_train, f_test, l_test)
f_train, l_train, f_test, l_test = feature_extraction(bike5, f_train, l_train, f_test, l_test)
f_train = f_train[1:, :, :]
f_test = f_test[1:, :, :]

In [8]:
return_shapes()

((388, 250, 9), (388,), (46, 250, 9), (46,))

In [9]:
f_train, l_train, f_test, l_test = feature_extraction(bus1, f_train, l_train, f_test, l_test)
f_train, l_train, f_test, l_test = feature_extraction(bus2, f_train, l_train, f_test, l_test)

In [10]:
return_shapes()

((573, 250, 9), (573,), (68, 250, 9), (68,))

In [11]:
f_train, l_train, f_test, l_test = feature_extraction(car1, f_train, l_train, f_test, l_test)
f_train, l_train, f_test, l_test = feature_extraction(car2, f_train, l_train, f_test, l_test)
f_train, l_train, f_test, l_test = feature_extraction(car3, f_train, l_train, f_test, l_test)

In [12]:
return_shapes()

((1070, 250, 9), (1070,), (124, 250, 9), (124,))

In [13]:
f_train, l_train, f_test, l_test = feature_extraction(tram1, f_train, l_train, f_test, l_test)
f_train, l_train, f_test, l_test = feature_extraction(tram2, f_train, l_train, f_test, l_test)

In [14]:
return_shapes()

((1165, 250, 9), (1165,), (136, 250, 9), (136,))

In [15]:
f_train, l_train, f_test, l_test = feature_extraction(train, f_train, l_train, f_test, l_test)

In [16]:
return_shapes()

((1260, 250, 9), (1260,), (147, 250, 9), (147,))

In [17]:
f_train, l_train, f_test, l_test = feature_extraction(walk1, f_train, l_train, f_test, l_test)
f_train, l_train, f_test, l_test = feature_extraction(walk2, f_train, l_train, f_test, l_test)
f_train, l_train, f_test, l_test = feature_extraction(walk3, f_train, l_train, f_test, l_test)
f_train, l_train, f_test, l_test = feature_extraction(walk4, f_train, l_train, f_test, l_test)
f_train, l_train, f_test, l_test = feature_extraction(walk5, f_train, l_train, f_test, l_test)

In [18]:
return_shapes()

((2202, 250, 9), (2202,), (254, 250, 9), (254,))

In [19]:
l_train = tf.one_hot(l_train, 6)

In [20]:
l_test = tf.one_hot(l_test, 6)

In [21]:
# parameters
learning_rate = 0.01
epochs = 1000

# network Parameters
n_classes = 6
dropout = 0.8

weights = {
    'wc1': tf.Variable(tf.random_normal([5, 9, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 32, 64])),
    'wc3': tf.Variable(tf.random_normal([5, 64, 128])),
    'wd1': tf.Variable(tf.random_normal([10*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bc3': tf.Variable(tf.random_normal([128])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

def conv1d(x, W, b):
    x = tf.nn.conv1d(x, W, 5, padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv1d(x, weights['wc1'], biases['bc1'])

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv1d(conv1, weights['wc2'], biases['bc2'])
    

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

x = tf.placeholder(tf.float32, [None, 250, 9])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        sess.run(optimizer, feed_dict={
            x: f_train,
            y: l_train.eval(),
            keep_prob: dropout})

            # Calculate batch loss and accuracy
        loss = sess.run(cost, feed_dict={
            x: f_train,
            y: l_train.eval(),
            keep_prob: 1.})

        print('Epoch {:>2}-'
            'Loss: {:>10.4f}'.format(
            epoch + 1,
            loss))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: f_test,
        y: l_test.eval(),
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1-Loss: 2845218058945953792.0000
Epoch  2-Loss: 72938516139440114499584.0000
Epoch  3-Loss: 2764003927081289630351360.0000
Epoch  4-Loss: 2563935043584.0000
Epoch  5-Loss: 504226686649237504.0000
Epoch  6-Loss: 273247262480531456.0000
Epoch  7-Loss: 42380701462429696.0000
Epoch  8-Loss:  3584.1001
Epoch  9-Loss:   829.3865
Epoch 10-Loss:   137.9806
Epoch 11-Loss:    50.4624
Epoch 12-Loss:    39.8983
Epoch 13-Loss:    33.0656
Epoch 14-Loss:    31.5604
Epoch 15-Loss:    29.8681
Epoch 16-Loss:    28.1193
Epoch 17-Loss:    26.7475
Epoch 18-Loss:    25.7219
Epoch 19-Loss:    24.7544
Epoch 20-Loss:    23.8052
Epoch 21-Loss:    22.8325
Epoch 22-Loss:    21.9337
Epoch 23-Loss:    21.0348
Epoch 24-Loss:    20.2694
Epoch 25-Loss:    19.5101
Epoch 26-Loss:    18.6682
Epoch 27-Loss:    17.7972
Epoch 28-Loss:    17.0940
Epoch 29-Loss:    16.2944
Epoch 30-Loss:    15.6111
Epoch 31-Loss:    14.9932
Epoch 32-Loss:    14.3610
Epoch 33-Loss:    13.7483
Epoch 34-Loss:    13.1065
Epoch 35-Loss:    

Epoch 305-Loss:     1.5771
Epoch 306-Loss:     1.5770
Epoch 307-Loss:     1.5779
Epoch 308-Loss:     1.5787
Epoch 309-Loss:     1.5778
Epoch 310-Loss:     1.5764
Epoch 311-Loss:     1.5753
Epoch 312-Loss:     1.5781
Epoch 313-Loss:     1.5767
Epoch 314-Loss:     1.5767
Epoch 315-Loss:     1.5766
Epoch 316-Loss:     1.5771
Epoch 317-Loss:     1.5757
Epoch 318-Loss:     1.5784
Epoch 319-Loss:     1.5776
Epoch 320-Loss:     1.5759
Epoch 321-Loss:     1.5754
Epoch 322-Loss:     1.5736
Epoch 323-Loss:     1.5694
Epoch 324-Loss:     1.5693
Epoch 325-Loss:     1.5716
Epoch 326-Loss:     1.5735
Epoch 327-Loss:     1.5727
Epoch 328-Loss:     1.5729
Epoch 329-Loss:     1.5734
Epoch 330-Loss:     1.5726
Epoch 331-Loss:     1.5719
Epoch 332-Loss:     1.5731
Epoch 333-Loss:     1.5716
Epoch 334-Loss:     1.5687
Epoch 335-Loss:     1.5687
Epoch 336-Loss:     1.5663
Epoch 337-Loss:     1.5668
Epoch 338-Loss:     1.5653
Epoch 339-Loss:     1.5660
Epoch 340-Loss:     1.5665
Epoch 341-Loss:     1.5657
E

Epoch 609-Loss:     1.5116
Epoch 610-Loss:     1.5118
Epoch 611-Loss:     1.5115
Epoch 612-Loss:     1.5115
Epoch 613-Loss:     1.5121
Epoch 614-Loss:     1.5115
Epoch 615-Loss:     1.5112
Epoch 616-Loss:     1.5108
Epoch 617-Loss:     1.5100
Epoch 618-Loss:     1.5096
Epoch 619-Loss:     1.5095
Epoch 620-Loss:     1.5092
Epoch 621-Loss:     1.5101
Epoch 622-Loss:     1.5099
Epoch 623-Loss:     1.5108
Epoch 624-Loss:     1.5113
Epoch 625-Loss:     1.5104
Epoch 626-Loss:     1.5103
Epoch 627-Loss:     1.5097
Epoch 628-Loss:     1.5088
Epoch 629-Loss:     1.5099
Epoch 630-Loss:     1.5112
Epoch 631-Loss:     1.5108
Epoch 632-Loss:     1.5102
Epoch 633-Loss:     1.5100
Epoch 634-Loss:     1.5089
Epoch 635-Loss:     1.5091
Epoch 636-Loss:     1.5095
Epoch 637-Loss:     1.5083
Epoch 638-Loss:     1.5077
Epoch 639-Loss:     1.5071
Epoch 640-Loss:     1.5075
Epoch 641-Loss:     1.5072
Epoch 642-Loss:     1.5071
Epoch 643-Loss:     1.5082
Epoch 644-Loss:     1.5075
Epoch 645-Loss:     1.5071
E

Epoch 913-Loss:     1.4921
Epoch 914-Loss:     1.4924
Epoch 915-Loss:     1.4917
Epoch 916-Loss:     1.4918
Epoch 917-Loss:     1.4920
Epoch 918-Loss:     1.4921
Epoch 919-Loss:     1.4918
Epoch 920-Loss:     1.4918
Epoch 921-Loss:     1.4921
Epoch 922-Loss:     1.4925
Epoch 923-Loss:     1.4919
Epoch 924-Loss:     1.4920
Epoch 925-Loss:     1.4917
Epoch 926-Loss:     1.4918
Epoch 927-Loss:     1.4914
Epoch 928-Loss:     1.4916
Epoch 929-Loss:     1.4915
Epoch 930-Loss:     1.4916
Epoch 931-Loss:     1.4910
Epoch 932-Loss:     1.4912
Epoch 933-Loss:     1.4916
Epoch 934-Loss:     1.4915
Epoch 935-Loss:     1.4914
Epoch 936-Loss:     1.4918
Epoch 937-Loss:     1.4919
Epoch 938-Loss:     1.4913
Epoch 939-Loss:     1.4906
Epoch 940-Loss:     1.4903
Epoch 941-Loss:     1.4905
Epoch 942-Loss:     1.4904
Epoch 943-Loss:     1.4912
Epoch 944-Loss:     1.4904
Epoch 945-Loss:     1.4908
Epoch 946-Loss:     1.4909
Epoch 947-Loss:     1.4915
Epoch 948-Loss:     1.4913
Epoch 949-Loss:     1.4910
E